In [26]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

dataset_path = '/Users/rjaditya/Documents/NEU-SEM/Spring-2024/ML/LLM-vs-Human-Text-Detection/dataset/train_v2_drcat_02.csv'
df = pd.read_csv(dataset_path, header = 0)
### get the labels
label = df['label']
# df.drop(columns='label', axis = 1, inplace = True)
df.drop(columns='RDizzl3_seven', axis = 1, inplace=True)
df.drop(columns='prompt_name', axis = 1, inplace=True)
df.drop(columns='source', axis = 1, inplace=True)

X_train, Y_train, X_test, Y_test = train_test_split(df, label, shuffle = True)

ds1 = pd.concat([X_train, Y_train])
ds2 = pd.concat([X_test, Y_test])
ds1.head()

,text,label
31052,The legendary entrepreneur and philanthropist ...,1
36091,I believe it is incredibly beneficial for old...,1
28491,"In recent years, there has been a growing tren...",1
10238,"In my opinion, the Facial Action Coding System...",0
16811,Driverless cars will be an important and inter...,0


In [28]:
train_ds = Dataset.from_pandas(ds1)
test_ds = Dataset.from_pandas(ds2)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/ML/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/fy/t6h9m9v57sb6bq5hmv_wv3zm0000gn/T/ipykernel_12154/474911649.py", line 2, in <module>
    test_ds = Dataset.from_pandas(ds2)
  File "/opt/homebrew/anaconda3/envs/ML/lib/python3.8/site-packages/datasets/arrow_dataset.py", line 861, in from_pandas
    table = InMemoryTable.from_pandas(
  File "/opt/homebrew/anaconda3/envs/ML/lib/python3.8/site-packages/datasets/table.py", line 720, in from_pandas
    return cls(pa.Table.from_pandas(*args, **kwargs))
  File "pyarrow/table.pxi", line 3874, in pyarrow.lib.Table.from_pandas
  File "/opt/homebrew/anaconda3/envs/ML/lib/python3.8/site-packages/pyarrow/pandas_compat.py", line 570, in dataframe_to_arrays
    convert_fields) = _get_columns_to_convert(df, schema, preserve_index,
  File "/opt/homebrew/anaconda3/envs/

In [8]:
from transformers import AlbertTokenizer, AlbertModel
import torch

### testing
model = AlbertModel.from_pretrained("albert-base-v2")
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

In [ ]:
import torch
import torch.nn.functional as F

batch = tokenizer(list(X_train['text']), padding = True, truncation = True, max_length = 1000, return_tensors = "pt")
print(batch)

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments("LLMvHuman")
trainer = Trainer(
    model,
    args,
    train_dataset=X_train['text'],
    eval_dataset=X_test['text'],
    data_collator=data_collector,
    tokenizer=tokenizer
)

while torch.no_grad():
    op = model(**batch)
    prediction = F.softmax(op.logits, dim = 1)
    labels = torch.argmax(prediction)
    print(label)